### Airnology 2023

#### Descriptions

**datetime**            : Waktu ketika data dihitung (dalam format timestamp UNIX).

**datetime_iso**        : Waktu dalam format ISO 8601, termasuk zona waktu.

**time-zone**           : Zona waktu dalam detik terhadap UTC.

**temp**                : Suhu saat ini dalam Celcius.

**visibility**          : Visibilitas rata-rata dalam meter.

**d_point**             : Titik embun saat ini dalam Celcius.

**feels**               : Suhu yang dirasakan saat ini dalam Celcius.

**min_temp**            : Suhu minimum dalam rentang waktu tertentu dalam Celcius.

**max_temp**            : Suhu maksimum dalam rentang waktu tertentu dalam Celcius.

**pressure**            : Tekanan atmosfer dalam hPa .

**sea_level**           : Tekanan atmosfer pada permukaan laut dalam hPa.

**grnd_level**          : Tekanan atmosfer pada permukaan tanah dalam hPa.

**hum**                 : Persentase kelembaban udara saat ini.

**wind_spd**            : Kecepatan angin saat ini dalam m/s.

**wind_deg**            : Arah angin dalam derajat.

**rain_1h**             : Curah hujan dalam 1 jam terakhir dalam mm. (variabel target)

**rain_3h**             : Curah hujan dalam 3 jam terakhir dalam mm.

**snow_1h**             : Curah salju dalam 1 jam terakhir dalam mm.

**snow_3h**             : Curah salju dalam 3 jam terakhir dalam mm.

**clouds**              : Persentase penutupan awan saat ini.

#### Libraries

In [17]:
import pandas as pd
import numpy as np

import datetime

#### Methods

In [58]:
def clean_temp(temp) :
    if isinstance(temp, str) :
        temp = temp.replace(' Celcius', '')
        temp = temp.replace(' C', '')
        temp = temp.replace('°C', '')
    return temp

#### Data Overview

In [3]:
train = pd.read_csv('../../datasets/train.csv')
test = pd.read_csv('../../datasets/test.csv')

In [12]:
print(f'train shape : {train.shape}')
print(f'test shape : {test.shape}')

train shape : (341880, 20)
test shape : (49368, 19)


#### Merging train and test

In [65]:
# merging train and test data
merged = pd.concat([train, test], axis = 0).reset_index(drop=True) 
merged.drop('datetime', axis=1, inplace=True)

# converting temp datatypes
for column in ['temp','d_point','feels','min_temp','max_temp'] :
    merged[column] = merged[column].apply(lambda x: clean_temp(x))
    merged[column] = merged[column].astype('float64')


In [66]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391248 entries, 0 to 391247
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   datetime_iso  391248 non-null  object 
 1   time-zone     391248 non-null  int64  
 2   temp          391248 non-null  float64
 3   visibility    58645 non-null   object 
 4   d_point       391247 non-null  float64
 5   feels         391248 non-null  float64
 6   min_temp      391248 non-null  float64
 7   max_temp      391248 non-null  float64
 8   prssr         391248 non-null  object 
 9   sea_level     220658 non-null  object 
 10  grnd_level    220785 non-null  object 
 11  hum           391248 non-null  object 
 12  wind_spd      391248 non-null  object 
 13  wind_deg      391248 non-null  object 
 14  rain_1h       341880 non-null  object 
 15  rain_3h       220170 non-null  object 
 16  snow_1h       220362 non-null  object 
 17  snow_3h       220502 non-null  object 
 18  clou